In [3]:
#main code fitting model and data
import pandas as pd
from gurobipy import Model, GRB, quicksum
from itertools import product
import json

#Input data
SHIPMENTS_XLSX = r"C:\Users\bhara\Documents\Airlink project\Shipment2025.xlsx"
SHIPMENTS_SHEET = 0
AIRPORTS_XLSX  = r"C:\Users\bhara\Documents\Airlink project\Regions.xlsx"
AIRPORTS_SHEET = 0

#Loading data
sh = pd.read_excel(SHIPMENTS_XLSX, sheet_name=SHIPMENTS_SHEET)
sh = sh.rename(columns={"AW (kg)":"AW_kg", "Aircraft Type":"Aircraft_Type"})
for col in ["Origin","Destination","AW_kg","Demand"]:
    if col not in sh.columns:
        raise ValueError(f"Missing column in shipments: {col}")
sh["AW_kg"]  = pd.to_numeric(sh["AW_kg"],  errors="coerce").fillna(0.0)
sh["Demand"] = pd.to_numeric(sh["Demand"], errors="coerce").fillna(0.0)

ap = pd.read_excel(AIRPORTS_XLSX, sheet_name=AIRPORTS_SHEET)
ap = ap.rename(columns={"AirportCode":"IATA"})  # no-op if already IATA
for col in ["Airport","Region"]:
    if col not in ap.columns:
        raise ValueError(f"Missing column in airports file: {col}")
ap_map = ap.set_index("Airport")["Region"].to_dict()

#Destination
sh["DestRegion"] = sh["Destination"].map(ap_map)
if sh["DestRegion"].isna().any():
    missing = sh.loc[sh["DestRegion"].isna(), "Destination"].unique()
    raise ValueError(f"These Destination codes have no Region mapping: {missing}")

#Aircraft capacity is the total weight transferred by the aircraft
sh["b_row_kg"] = sh["AW_kg"]

def solve_one(alpha: float, c_cap: float, c_unmet: float, output_flag: int = 0):
    """
    Row-level leader model:
      (1a) min sum l_i
      (1b) (1-alpha) l_i >= alpha * c_cap * k_i
      (1c) alpha l_i >= c_unmet (1-alpha) (Demand_i - k_i)
      (1d) k_i <= b_i
    Returns (status, objective, sum_k, sum_l, alloc_df or None)
    """
    m = Model(f"Leader_Row_alpha{alpha:.2f}_cc{c_cap:.1f}_cu{c_unmet:.1f}")
    m.Params.OutputFlag = output_flag

    k = {i: m.addVar(vtype=GRB.INTEGER,    lb=0.0, name=f"k[{i}]") for i in sh.index}
    l = {i: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"l[{i}]") for i in sh.index}
    m.update()

    m.setObjective(quicksum(l.values()), GRB.MINIMIZE)  # (1a)

    for i in sh.index:  # (1b)
        m.addConstr((1.0 - alpha) * l[i] >= alpha * c_cap * k[i], name=f"tradeoff[{i}]")

    for i, r in sh.iterrows():  # (1c) row-level
        m.addConstr(alpha * l[i] >= c_unmet * (1.0 - alpha) * (float(r["Demand"]) - k[i]),
                    name=f"unmet_row[{i}]")

    for i, r in sh.iterrows():  # (1d) caps
        m.addConstr(k[i] <= float(r["b_row_kg"]), name=f"cap_row[{i}]")

    m.optimize()

    alloc = pd.DataFrame([{
        "row_id": i,
        "Origin": sh.at[i, "Origin"],
        "Destination": sh.at[i, "Destination"],
        "DestRegion": sh.at[i, "DestRegion"],
        "AW_kg(b_i)": float(sh.at[i, "b_row_kg"]),
        "Demand_i": float(sh.at[i, "Demand"]),
        "k_alloc_kg": float(k[i].X),
        "l_value": float(l[i].X),
    } for i in sh.index])

    obj   = float(m.ObjVal)
    sum_k = float(alloc["k_alloc_kg"].sum())
    sum_l = float(alloc["l_value"].sum())
    return obj, sum_k, sum_l, alloc

#parameters
ALPHAS   = [round(i/10, 1) for i in range(1, 10)]       #alpha from 0.1 to 0.9
C_CAPS   = list(range(1, 11))                           #cost of capacity from $1 to $10
C_UNMETS = list(range(1, 11))                           #cost of unmet demand from $1 to $10

rows = []
best = {"objective": float("inf")}
for a, cc, cu in product(ALPHAS, C_CAPS, C_UNMETS):
    obj, sum_k, sum_l, alloc = solve_one(a, cc, cu, output_flag=0)

    rows.append({
        "alpha": a,
        "c_capacity_per_kg": cc,
        "c_unmet_per_kg": cu,
        "sum_Demand_kg": float(sh["Demand"].sum()),
        "sum_k_alloc_kg": sum_k,
        "objective": obj,
    })

    """ if pd.notna(obj) and obj < best["objective"]:
        best = {"alpha": a, "c_capacity_per_kg": cc, "c_unmet_per_kg": cu,
                "objective": obj, "sum_k_alloc_kg": sum_k}
        # Save current best allocation
        if alloc is not None:
            outname = f"best_alloc_alpha{a:.1f}_cc{cc}_cu{cu}.csv"
            alloc.to_csv(outname, index=False)"""

#results table
sweep = pd.DataFrame(rows).sort_values(["alpha","c_capacity_per_kg","c_unmet_per_kg"])

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)
pd.set_option("display.float_format", "{:,.4f}".format)

print(
    sweep[
        ["alpha", "c_capacity_per_kg", "c_unmet_per_kg",
         "objective", "sum_Demand_kg", "sum_k_alloc_kg"]
    ].to_string(index=False)
)

print(f"\nTotal combinations evaluated: {len(sweep)}")




 alpha  c_capacity_per_kg  c_unmet_per_kg      objective  sum_Demand_kg  sum_k_alloc_kg
0.1000                  1               1   565,314.7639   690,670.7515    627,858.0000
0.1000                  1               2 1,130,629.5277   690,670.7515    627,858.0000
0.1000                  1               3 1,695,944.2916   690,670.7515    627,858.0000
0.1000                  1               4 2,261,259.0554   690,670.7515    627,858.0000
0.1000                  1               5 2,826,573.8193   690,670.7515    627,858.0000
0.1000                  1               6 3,391,888.5832   690,670.7515    627,858.0000
0.1000                  1               7 3,957,203.3470   690,670.7515    627,858.0000
0.1000                  1               8 4,522,518.1109   690,670.7515    627,858.0000
0.1000                  1               9 5,087,832.8747   690,670.7515    627,858.0000
0.1000                  1              10 5,653,147.6386   690,670.7515    627,858.0000
0.1000                  2       